In [36]:
import pandas as pd

# Constants
INTERNAL_AUDIENCE_SIZE_FILE = 'seed_to_audience_per_1k.csv'  # Path to your internal CSV file

# Load internal audience sizes file
def load_file(file_path):
    try:
        data = pd.read_csv(file_path)
        print("Column names in the internal file:", data.columns.tolist())  # Display column names for debugging
        return data
    except Exception as e:
        print(f"Failed to load internal data: {str(e)}")
        return None

In [60]:
data= pd.read_csv('seed_to_audience_per_1k.csv')
data

,SEED SIZE,AUDIENCE SIZE APROX
0,"25,000","1,350,000"
1,"26,000","1,364,000"
2,"27,000","1,378,000"
3,"28,000","1,392,000"
4,"29,000","1,406,000"
...,...,...
10971,"10,996,000","95,968,000"
10972,"10,997,000","95,976,000"
10973,"10,998,000","95,984,000"
10974,"10,999,000","95,992,000"


In [76]:
seed_size = 273673
tolerance=0.01
# Convert seed_size to integer if it's input as a string with commas
seed_size = int(seed_size.replace(',', '')) if isinstance(seed_size, str) else seed_size 
    # Convert data columns to integer for proper calculation

#data['SEED SIZE'] = data['SEED SIZE'].str.replace(',', '').astype(int)
#data['AUDIENCE SIZE APROX'] = data['AUDIENCE SIZE APROX'].str.replace(',', '').astype(int)

data


,SEED SIZE,AUDIENCE SIZE APROX,Percent Difference
0,25000,1350000,0.776786
1,26000,1364000,0.767857
2,27000,1378000,0.758929
3,28000,1392000,0.750000
4,29000,1406000,0.741071
...,...,...,...
10971,10996000,95968000,97.178571
10972,10997000,95976000,97.187500
10973,10998000,95984000,97.196429
10974,10999000,95992000,97.205357


In [77]:

# Calculate the percent difference between each seed size and the input seed size
data['Percent Difference'] = abs(data['SEED SIZE'] - seed_size) / seed_size
    

data


,SEED SIZE,AUDIENCE SIZE APROX,Percent Difference
0,25000,1350000,0.908650
1,26000,1364000,0.904996
2,27000,1378000,0.901342
3,28000,1392000,0.897688
4,29000,1406000,0.894034
...,...,...,...
10971,10996000,95968000,39.179338
10972,10997000,95976000,39.182992
10973,10998000,95984000,39.186646
10974,10999000,95992000,39.190300


In [78]:
 # Find the index of the smallest percent difference that is within the tolerance
closest_size_index = data[data['Percent Difference'] <= tolerance]['Percent Difference'].idxmin()
print(closest_size_index)

    # Check if a closest size was found within the tolerance
if pd.notna(closest_size_index):
    closest_size = data.loc[closest_size_index, 'AUDIENCE SIZE APROX']
    closest_size
else:
    closest_size_index = data['Percent Difference'].idxmin()
    data.loc[closest_size_index, 'AUDIENCE SIZE APROX']    
    # If no size is within the tolerance, return the closest size ignoring the tolerance



249


In [85]:
data.loc[closest_size_index, 'AUDIENCE SIZE APROX']    
dry_run_data = pd.read_csv('Dry run - 36328.csv')
dry_run_data

,threshold,percentile_sum,cumulative_sum
0,0.1050,1806446,1806446
1,0.1049,5729,1812175
2,0.1048,5617,1817792
3,0.1047,5992,1823784
4,0.1046,5845,1829629
...,...,...,...
1045,0.0005,52548,691431264
1046,0.0004,19276,691450540
1047,0.0003,5595,691456135
1048,0.0002,1021,691457156


In [80]:
tolerance=0.01
# Ensure cumulative_sum is converted to a float and the Rounded Cumulative Sum is calculated
dry_run_data['cumulative_sum'] = dry_run_data['cumulative_sum'].astype(float)
#dry_run_data['Rounded Cumulative Sum'] = dry_run_data['cumulative_sum'].round(-4)  # Round to nearest 10,000
# Calculate the percent difference between each cumulative_sum and the closest_size
dry_run_data['Percent Difference'] = abs(dry_run_data['cumulative_sum'] - closest_size) / closest_size
    
    # Find the index of the smallest percent difference that is within the tolerance
valid_indices = dry_run_data['Percent Difference'] <= tolerance
if valid_indices.any():  # Check if there's any entry within the tolerance
    closest_row_index = dry_run_data.loc[valid_indices, 'Percent Difference'].idxmin()
    print(closest_row_index)
else:
    # If no entry is within the tolerance, find the closest one ignoring the tolerance
    closest_row_index = dry_run_data['Percent Difference'].idxmin()
# Return the threshold of the closest row
threshold = dry_run_data.loc[closest_row_index, 'threshold']
print(threshold)

236
0.0814


In [81]:
dry_run_data

,threshold,percentile_sum,cumulative_sum,Percent Difference
0,0.1050,1806446,1806446.0,0.587946
1,0.1049,5729,1812175.0,0.586639
2,0.1048,5617,1817792.0,0.585358
3,0.1047,5992,1823784.0,0.583991
4,0.1046,5845,1829629.0,0.582658
...,...,...,...,...
1045,0.0005,52548,691431264.0,156.716985
1046,0.0004,19276,691450540.0,156.721382
1047,0.0003,5595,691456135.0,156.722659
1048,0.0002,1021,691457156.0,156.722891


In [82]:
audience_stats_data = pd.read_csv('Audience statistic - 36328.csv')

stats = audience_stats_data[audience_stats_data['SCORE_THRESHOLD'] == threshold]
idx = stats.index[0]
stats_2 =  audience_stats_data.loc[idx-2:idx+2]
print(stats_2)
#if not stats.empty:
#    idx = stats.index[0]
#    return audience_stats_data.loc[idx-2:idx+2]  # Include two rows above and below the matched row
#return None

     SCORE_THRESHOLD  AUDIENCE_SIZE  VERIFIED_PATIENTS  AQ_SCORE  \
234           0.0816        2512200             150300  0.059828   
235           0.0815        2523500             150350  0.059580   
236           0.0814        2534500             150350  0.059321   
237           0.0813        2545500             150400  0.059085   
238           0.0812        2556550             150450  0.058849   

     INCREMENTAL_AQ_SCORE  PERCENT_POPULATION  PERCENT_PATIENTS  THRESHOLD_3X  \
234              0.004587              0.5608           82.3562      0.001222   
235              0.004425              0.5633           82.3836      0.001222   
236              0.000000              0.5657           82.3836      0.001222   
237              0.004545              0.5682           82.4110      0.001222   
238              0.004525              0.5707           82.4384      0.001222   

     PREVALENCE  MODEL_POWER_INDEX  TRUE_POSITIVE  FALSE_POSITIVE  \
234    0.000407         146.997543 

In [87]:
data.iloc[249]

SEED SIZE              2.740000e+05
AUDIENCE SIZE APROX    4.384000e+06
Percent Difference     1.194857e-03
Name: 249, dtype: float64